# Gradient Descent

How do we find the optimal weights for our learning models? Minimize the cost function. But how do we find the minimum? Gradient descent.

In [ ]:
# load in our libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

#### But first our data

location: `/src/data/apes.csv`

Our data for today is just a few gorillas and what we have measure is each height and weight. 

In [ ]:
data = "../data/apes.csv"
gorillas = pd.read_csv(data, index_col="id")
gorillas.head()

What our task is is to build a model that predicts a gorilla's height from their weight. Therefore, our `x` is weight and our `y` is height.

In [ ]:
X = gorillas['weight'].values.reshape(-1,1)
y = gorillas['height'].values

plt.scatter(X, y)
plt.xlabel('weight(kg)')
plt.ylabel('height(cm)')

## Finding the model parameters with linear regression

Let's use least squares to find the optimal parameters (slope and intercept) for the simple linear regression model. 

When we run this following bit of code, we get the following equation for our model.

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X,y)
model.intercept_
model.coef_
print("y = {:.4f}x + {:.4f}".format(model.coef_[0], model.intercept_))

In [ ]:
# X = weight
# y = height
plt.scatter(X, y)
plt.plot(X, model.predict(X))
plt.xlabel('weight(kg)')
plt.ylabel('height(cm)')


But let's play around with this to get a better intuition at how we derive these optimal values. To do so, we will only attempt to learn the optimal intercept. We'll rely on our model to have given us the optimal slope and will from here on forward hold this parameter constant at 0.4117.

What if we develop our own model where attempt to find the best intercept. How would we go about doing that? 

In [ ]:
## first create a linear model that takes in the gorilla's weight,
## a slope (which we will always pass th value 0.4117 to
## and an intercept which we can change to see if we can minimize the loss function
def our_model(weight, slope, intercept):
    height = slope * weight + intercept
    return height

## let's test it out
weight = 150
slope = model.coef_[0]
our_intercept = 80

our_model(weight, slope, our_intercept)

Now let's run this model over our entire dataset for each gorilla's weight to predict each gorilla's height given our *randomly* assigned intercept.  We'll use these values to see what our new function looks like...

In [ ]:
our_predictions = our_model(gorillas['weight'], slope, our_intercept)
our_predictions

plt.scatter(X, y)
plt.plot(X, model.predict(X), alpha=0.3)
plt.plot(X, our_predictions.values.reshape(-1,1))
plt.xlabel('weight(kg)')
plt.ylabel('height(cm)')

Let's us now calculate the residual sum of squares (rss/sse) as our loss. 

In [ ]:
residuals = y - our_predictions
squared_residuals = residuals**2
sse = np.sum(squared_residuals)
print("sse: ", sse)

And now let's plot that loss (72.845) against the intercept (80) that resulted in that loss.

In [ ]:
plt.scatter(our_intercept, sse)
plt.xlabel('intercept')
plt.ylabel('loss (sse)')

Now, let's pull this code into a function so that we can give ourself a simple way to calculate the loss given our observed weights (inputs), our observed heights (target) and any intercept. Essentially, since we want a way to minimize this loss, it is nice to wrap this logic in a function.

In [ ]:
slope = model.coef_[0] # a refresher that our slope came from our linear regressor
def calculate_loss(x, y, intercept):
    predictions = our_model(x, slope, intercept)
    residuals = y - predictions
    squared_residuals = residuals**2
    sse = np.sum(squared_residuals)
    return sse

Now we just need to play around to see how this function works. Let's go ahead and try to intercepts and see how that affects the loss.

In [ ]:
old_intercept = 80
new_intercept = 81
old_loss = calculate_loss(gorillas['weight'], gorillas['height'], old_intercept)
new_loss = calculate_loss(gorillas['weight'], gorillas['height'], new_intercept)

print("old intercept {} and old loss {}".format(old_intercept, old_loss))
print("new intercept {} and new loss {}".format(new_intercept, new_loss))

## Finding the mimimum loss

First thing that would be good to do is to remind ourself of the equation for our model so we can actually see what the optimal intercept is.

The next thing that I want to do is brute force the process of finding the optimal intercept. Essentially, slowly increase the intercept to descend down the loss function toward some minimum until we find the lowest point. Let's plot what I am saying...

In [ ]:
print("y = {:.4f}x + {:.4f}".format(model.coef_[0], model.intercept_))

brute_intercepts = np.linspace(80, 84)
losses = np.array([calculate_loss(gorillas['weight'], gorillas['height'], interc) for interc in brute_intercepts])

# grab the first guess intercept and the known optimal
sample_intercepts = np.array([80, model.intercept_])
sample_losses = np.array([calculate_loss(gorillas['weight'], gorillas['height'], interc) for interc in sample_intercepts])


plt.plot(brute_intercepts, losses)
plt.scatter(sample_intercepts, sample_losses) 
plt.xlabel('intercept')
plt.ylabel('loss (sse)')

But this is expensive!!! Remember, we are only trying to learn one parameter at the momemnt. Some our models in the future might have hundreds!!! Let's calculate the direction in which we should move down this function. In other words, we want to find the derivative of the loss function with respect to the intercept. 

In other words, this is the reate of change with respect to any given intercept that we choose... in other other words the slope of the function at that point.

$ {residual} = ({observed_{height}} - {predicted_{height}})^2 $

$ {predicted_{height}} = {0.4117}{weight} + {intercept} $

$ {residual} = ({observed_{height}} - {0.4117}{weight} + {intercept})^2 $

$ sse = \sum{residual_i}$

So we have to take the derivative of the Sum of Squared Errors with respect to the intercept which is just the sum of the derivatives with respect to the intercept of each square error.

Let's demonstrate!

In [ ]:
# derivative allows us to calculate the numeric derivative
from scipy.misc import derivative

Let's take the first gorilla as an example and plug in those values to find our derivative given a particular intercept.

In [ ]:
# numeric derivative
observed_height = gorillas['height'].loc[1]
weight = gorillas['weight'].loc[1]


def calculate_residual(intercept):
    return (observed_height - our_model(weight, slope, intercept))**2


derivative(calculate_residual, 82.0468)


Now we need to sum these derivatives to find the derivative of the residual sum of squares function with respect to the intercept. The function below just finds all of the derivatives for each observed weight, observed height and a given intercept.

In [ ]:
def find_derivatives(x, y, intercept, print_output):
    derivatives = []
    for i in range(gorillas.shape[0]):
        observed_height = gorillas['height'].loc[i + 1]
        weight = gorillas['weight'].loc[i + 1]
        
        def calculate_residual(intercept):
            return (observed_height - our_model(weight, slope, intercept))**2
        
        d = derivative(calculate_residual, intercept)
        if print_output: 
            print("observation: {} (observed height: {:.4f}; weight: {:.4f})".format(i + 1, observed_height, weight))
            print("equation: ({:.4f} - ({:.4f} X {:.4f} + intercept))^2".format(observed_height, slope, weight))
            print("derivative: {}\n".format(d))
        derivatives.append(d)
        
    return np.array(derivatives)

And now let's try it out for our intercept 80.

In [ ]:
ds = find_derivatives(gorillas['weight'], gorillas['height'], 80, True)
print('individual residual function derivs: ', ds)
print('resid sum square function deriv: ', sum(ds))

The neat thing is, we know we are at a minimum point when the derivative approaches 0. Here, we know our optimal intercept is around 82.0468. Let's see if the slope of the line is approaching flatness if we hand our newly created function this value (just to test our intuition).

In [ ]:
ds = find_derivatives(gorillas['weight'], gorillas['height'], 82.0468, True)
print(ds)
print(sum(ds))

Perhaps, it is easier to plot this just to see what is happening here.

In [ ]:
# just a helper function
def solve_for_b(slope,x, y):
    b = (slope * x - y)/-1
    return b

ix = 82.0468 # we can change this value to see the slope at different intercepts along this function
iy = calculate_loss(gorillas['weight'], gorillas['height'], ix)
m = sum(find_derivatives(gorillas['weight'], gorillas['height'], ix, False))
b = solve_for_b(m, ix, iy)

def get_point(m, x, b):
    return m * x + b

tmp_x = np.array([ix-0.5, ix+0.5])
tmp_y = get_point(m, tmp_x, b)

tmp_y


plt.plot(brute_intercepts, losses)
plt.scatter(sample_intercepts, sample_losses) 
plt.plot(tmp_x, tmp_y)
plt.xlabel('intercept')
plt.ylabel('loss (sse)')

## Gradient Descent!!!
We can now use this value to tell us what direction and how big of a step to make and now we just need to take a step in that direction (opposite of the sign of the derivative). To specify the step size, we will just multiple the derivative by a learning rate (we have to decide this). The larger the learning rate the faster we learn but the higher the probability we miss the maximum.

The nice thing however, is as our derivative become smaller the smaller our steps become which means we make more adjustments toward the minimum that we do if our loss is higher.

In [ ]:
start_intercept = 80
learning_rate = 0.1
direction = sum(find_derivatives(gorillas['weight'], gorillas['height'], start_intercept, False))
step_size = direction * learning_rate
print("step size ", step_size)

new_intercept = old_intercept - step_size
print("new intercept ", new_intercept)



And finally, let's find a better intercept by using gradient descent!!!

In [ ]:
def gradient_descent(start_intercept, learning_rate):
    old_intercept = start_intercept
    
    for i in range(40):
        direction = sum(find_derivatives(gorillas['weight'], gorillas['height'], old_intercept, False))
        step_size = direction * learning_rate
        print("step size ", step_size)
        new_intercept = old_intercept - step_size
        loss = calculate_loss(gorillas['weight'], gorillas['height'], new_intercept)
        print("new intercept ", new_intercept)
        print("loss: ", loss, "\n")
        old_intercept = new_intercept

In [ ]:
gradient_descent(80, 0.01)